# 0.导包、导数据

In [1]:
import tensorflow as tf
import model.vgg16_structure as vgg

# 1.网络参数

In [2]:
"""
size-format:
             [ [      convolution_kernel     ], [   bias   ] ]
             [ [ f_h, f_w, in_size, out_size ], [ out_size ] ]
"""
structure = {

    # convolution layer 1
    'conv1_1': [[3, 3, 3, 64], [64]],
    'conv1_2': [[3, 3, 64, 64], [64]],

    # convolution layer 2
    'conv2_1': [[3, 3, 64, 128], [128]],
    'conv2_2': [[3, 3, 128, 128], [128]],

    # convolution layer 3
    'conv3_1': [[3, 3, 128, 256], [256]],
    'conv3_2': [[3, 3, 256, 256], [256]],
    'conv3_3': [[3, 3, 256, 256], [256]],

    # convolution layer 4
    'conv4_1': [[3, 3, 256, 512], [512]],
    'conv4_2': [[3, 3, 512, 512], [512]],
    'conv4_3': [[3, 3, 512, 512], [512]],

    # convolution layer 5
    'conv5_1': [[3, 3, 512, 512], [512]],
    'conv5_2': [[3, 3, 512, 512], [512]],
    'conv5_3': [[3, 3, 512, 512], [512]],

    # fully-connection 6
    'fc6': [[4096, 0, 0, 0], [4096]],

    # fully-connection 7
    'fc7': [[4096, 0, 0, 0], [4096]],

    # fully-connection 8
    'fc8': [[1000, 0, 0, 0], [1000]],

    # for cifar
    'cifar': [[512, 0, 0, 0], [512]],
}

# kernel_size is constant, so defined here
ksize = [1, 2, 2, 1,]

# convolution-layer-strides is already below
conv_strides = [1, 1, 1, 1,]

# pooling-layer-strides is already below
pool_strides = [1, 2, 2, 1,]

# 2.网络结构

In [ ]:
class Vgg16:
    def __init__(self):
        pass
    def build(self,input,is_training = True):
        self.conv1_1 = self.conv(input,'conv1_1')
        self.conv1_2 = self.conv(self.conv1_1,'conv1_2')
        self.pool1 = self.pool(self.conv1_2, 'pool1')
        
        self.conv2_1 = self.conv(self.pool1,'conv2_1')
        self.conv2_2 = self.conv(self.conv2_1,'conv2_2')
        self.pool2 = self.pool(self.conv2_2,'pool2')
        
        self.conv3_1 = self.conv(self.pool2,'conv3_1')
        self.conv3_2 = self.conv(self.conv3_1,'conv3_2')
        self.conv3_3 = self.conv(self.conv3_2,'conv3_3')
        self.pool3 = self.pool(self.conv3_3,'pool3')
        
        self.conv4_1 = self.conv(self.pool3,'conv4_1')
        self.conv4_2 = self.conv(self.conv4_1,'conv4_2')
        self.conv4_3 = self.conv(self.conv4_2,,'conv4_3')
        self.pool4 = self.pool(self.conv4_3,'pool4')
        
        self.conv5_1 = self.conv(self.pool4,'conv5_1')
        self.conv5_2 = self.conv(self.conv5_1,'conv5_2')
        self.conv5_3 = self.conv(self.conv5_2,'conv5_3')
        self.pool5 = self.pool(self.conv5_3,'pool5')
        
        self.fc1 = self.fc()
        self.fc2 = self.fc()
        self.fc3 = self.fc()
        
        return self.fc3
    
    def conv(input,name):
        print('Current input size in convolution layer is: '+str(input.get_shape().as_list()))
        with tf.variable_scope(name):
            size = structure[name]
            
            kernel = self.get_weight(size[0],name = 'w_' + name)
            bias = self.get_bias(size[1],name = 'b_' + name)
            
            conv = tf.nn.conv2d(input,kernel,strides = conv_strides,padding = 'SAME',name = name)
            out = tf.nn.relu(tf.add(conv,bias))
        return self.batch_normalization(out)
    
    def pool(self,input,name):
        return tf.nn.max_pool(input,ksize = ksize,strides = pool_strides, padding='SAME', name=name)
    
    def fully_connection(self, input, name):
        size = structure[name]
        
        with tf.variable_scope(name):
            shape = input.get_shape().as_list()
            dim = reduce(lambda x,y:x * y,shape[1:])
            x = tf.reshape(input,[-1,dim])
            
            weights = self.get_weight([dim, size[0][0]], name=name)
            biases = self.get_bias(size[1], name=name)
            
            tf.nn.relu(tf.nn.bias_add(tf.matmul(x,weights),bias))

# 3.训练

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())